In [1]:
import os 
import streamlit as st
from langchain_groq import ChatGroq
from langchain_community.embeddings import OllamaEmbeddings 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS 
from langchain.chains import create_retrieval_chain
from langchain_community.document_loaders import PyPDFDirectoryLoader
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

In [4]:
groq_api_key = os.getenv('GROQ_API_KEY')

In [5]:
llm = ChatGroq(groq_api_key=groq_api_key, model_name = 'llama3-8b-8192')

In [6]:
prompt = ChatPromptTemplate.from_template(
    '''
    Answer the question based on the provided context only
    Please provide the most accurate response based on the question
    <context>
    {Context}
    <context>
    Question:{input}
    '''
)

In [8]:
def create_vector_embedding():
    if 'vector' not in st.session_state:
        st.session_state.embeddings = OllamaEmbeddings()
        st.session_state.loader = PyPDFDirectoryLoader("GroqAPI\research_papers")
        st.session_state.docs = st.session_state.loader.load()
        st.session_state.text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=2000)
        st.session_state.final_docs = st.session_state.text_splitter.split_documents(st.session_state.docs[:50])
        st.session_state.vectors = FAISS.from_documents(st.session_state.final_docs ,st.session_state.embeddings)
        
    st.title("RAG documents Q&A with Groq and LLama3")
    user_prompt = st.text_input("Enter your query from Research paper docs")
    if st.button("Documents Embedding"):
        create_vector_embedding()
        st.write("VectorDB is ready")
        
    import time 

    if user_prompt:
        document_chain = create_stuff_documents_chain(llm,prompt)
        retriever = st.session_state.vectors.as_retriever()
        retriever_chain = create_retrieval_chain(retriever, document_chain)
        start=time.process_time()
        response=retrieval_chain.invoke({'input':user_prompt})
        print(f"Response time :{time.process_time()-start}")

        st.write(response['answer'])

        ## With a streamlit expander
        with st.expander("Document similarity Search"):
            for i,doc in enumerate(response['context']):
                st.write(doc.page_content)
                st.write('------------------------')
